In [1]:
# Rode o código se precisar instalar o snscrape
%pip install git+https://github.com/JustAnotherArchivist/snscrape.git

# Rode esse código se precisar instalar o Pandas
# !pip install pandas

  Cloning https://github.com/JustAnotherArchivist/snscrape.git to /private/var/folders/zc/jm1zv2_94qs5k_5xkchcbd200000gn/T/pip-req-build-io6ngqi6
  Running command git clone --filter=blob:none --quiet https://github.com/JustAnotherArchivist/snscrape.git /private/var/folders/zc/jm1zv2_94qs5k_5xkchcbd200000gn/T/pip-req-build-io6ngqi6
  Resolved https://github.com/JustAnotherArchivist/snscrape.git to commit e13033fea0da6e64684513b2fbc489883698f28a
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 10.9 MB/s eta 0:00:0000:0100:01
  Created wheel for snscrape: filename=snscrape-0.4.3.20220107.dev63+ge13033f-py3-none-any.whl size=67978 sha256=3f25dda871e41296be13820b293b2fa95875ff51361e9a1531f8c04a2c2d4d78
  Stored in directory: /private/var/folders/zc/jm1zv2_94qs5k_5xkchcbd200000gn/T/pip-ephem-wheel-cache-cvysxs5w/wheels/1a/ba/e2/39fa3a11802c4a622f2efc8be3f5ff854481051d0b4c95c1fd
Successfully built snscrape
Note: you may need to restart the ker

In [2]:
# Imports
import snscrape.modules.twitter as sntwitter
import pandas as pd
import datetime as dt


In [9]:
def ScrapTweets(max_tweets: int, search_string: str):
    # Definindo o máximo de tweets
    maxTweets = max_tweets

    # Criando a lista para salvar os dados
    tweets_list = []

    # Usando o TwitterSearchScraper para fazer o scrape dos dados data e salvar em uma lista
    for i, tweet in enumerate(
        sntwitter.TwitterSearchScraper(search_string).get_items()
    ):
        if i > maxTweets:
            break
        tweets_list.append(
            [
                tweet.date,
                tweet.replyCount,
                tweet.retweetCount,
                tweet.likeCount,
                tweet.place,
                tweet.sourceLabel,
                tweet.rawContent,
                tweet.user.username,
                tweet.user.followersCount,
                tweet.user.friendsCount,
                tweet.user.created,
                tweet.user.verified,
                tweet.user.statusesCount,
                tweet.user.favouritesCount,
                tweet.user.listedCount,
                tweet.user.mediaCount,
            ]
        )
    # Criando um dataframe com os dados
    tweets_df = pd.DataFrame(
        tweets_list,
        columns=[
            "DataHora",
            "ReplyCount",
            "Retweet Cont",
            "LikeCount",
            "Local",
            "Source",
            "Texto",
            "Usuario",
            "NumSeguidores",
            "NumAmigos",
            "UsuarioCriado",
            "UsuarioVerificado",
            "StatusCount",
            "favouritesCount",
            "listedCount",
            "mediaCount",
        ],
    )
    # Adicionando a coluna de data, mês e hora
    tweets_df["dia"] = tweets_df.DataHora.dt.day
    tweets_df["mes"] = tweets_df.DataHora.dt.month
    tweets_df["ano"] = tweets_df.DataHora.dt.year

    return tweets_df


In [13]:
def FormatToIRAMUTEQ(tweets: pd.DataFrame):
    # Criando um dicionário a partir do dataframe
    tweets_dict = tweets.to_dict()

    with open("corpus.txt", "w", encoding="utf-8") as corpus:
        for i in range(len(tweets_dict["DataHora"])):
            corpus.write("**** ")
            corpus.write(f'*ano_{tweets_dict["ano"][i]} ')
            corpus.write(f'*mes_{tweets_dict["mes"][i]} ')
            corpus.write(f'*dia_{tweets_dict["dia"][i]} ')
            corpus.write(f'*usuario_{tweets_dict["Usuario"][i]} ')
            corpus.write(f'*fonte_{tweets_dict["Source"][1].replace(" ", "_")}\n')
            corpus.write(tweets_dict["Texto"][i].replace("\n", " ").replace("*", ""))
            corpus.write("\n\n")


In [ ]:
# Na string de busca você pode usar ou omitir os marcadores from e until
# Exemplo:  'rockinrio from:anitta until:2020-07-31'

tweets = ScrapTweets(1000, "COVID vacina cloroquina")

tweets.head()


In [15]:
# Exportar dataframe para CSV
tweets.to_csv("text-query-tweets.csv", sep=",", index=False)

# Criar corpus para IRAMUTEQ
FormatToIRAMUTEQ(tweets)